In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time

In [6]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# 애월읍 음식점 
aewol = ['고내리', '고성리', '곽지리', '광령리', '구엄리', '금성리', '납읍리', '봉성리', '상가리', '상귀리', '소길리', '수산리', '신엄리', '애월리', '어음리', '유수암리', '장전리', '하가리', '하귀1리', '하귀2리']
# aewol = ['고성리']
eup = '애월읍'


for ri in aewol : 
    url = f"https://map.naver.com/v5/search/{eup}+{ri}+음식점?c=14060546.9964590,3952164.0916793,9,0,0,0,dh"
    driver.get(url)
    driver.implicitly_wait(5)

    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
    time.sleep(2)
    
    # 페이지번호를 위한 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 몇번 돌지 정하기 
    a = soup.select('div.zRM9F > a ')#페이지 위치
    a = [int(i.text) for i in a[1:-1]]
    b = max(a)

    
    data = []

    for _ in range(b) :

        for i in range(5):
            scroll = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]')))
            driver.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", scroll)  
            time.sleep(3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        naver = soup.select('#_pcmap_list_scroll_container > ul > li')

        for store in naver:
            try: 
                name = store.select(" div >  span.place_bluelink TYaxT ")[0].text.strip() 
                try :
                    point = store.select('em')[0].text

                except : 
                    point = ""

                review = store.select('div._2nqHm')[0].text
                review = review.replace(",", "")

                data.append([name, point, review])
            except :
                pass

        driver.find_element(By.CSS_SELECTOR,f'#app-root > div > div._2lx2y > div._2ky45 > a:nth-child({b+2}) > svg').click()
        time.sleep(1)




    df = pd.DataFrame(data, columns = ['name', 'point', 'review']) 
    df['visit_review'] = df.review.str.extract(r'방문자리뷰\s(\d*)')
    df['blog_review'] = df.review.str.extract(r'블로그리뷰\s(\d*)')
    df = df.drop('review', axis = 1)
    
    
    # 카페데이터 불러오기 
    url = f"https://map.naver.com/v5/search/{eup}+{ri}+카페?c=14060546.9964590,3952164.0916793,9,0,0,0,dh"
    driver.get(url)
    driver.implicitly_wait(5)

    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
    time.sleep(2)
    
    # 페이지번호를 위한 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 몇번 돌지 정하기 
    try :
        a = soup.select('div._2ky45 > a ')
        a = [int(i.text) for i in a[1:-1]]
        b = max(a)
    except :
        pass
    
    
    caffe = []
    for _ in range(b) :
        for i in range(10):
            scroll = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]')))
            driver.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", scroll)  
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        naver = soup.select('#_pcmap_list_scroll_container > ul > li')

        
        for store in naver:
            try: 
                name = store.select(" div > div > span._3Apve")[0].text.strip() 
                caffe.append(name)
            except :
                pass
        try:
            driver.find_element(By.CSS_SELECTOR,f'#app-root > div > div._2lx2y > div._2ky45 > a:nth-child({b+2}) > svg').click()
            time.sleep(1)
        except :
            pass
        
    # 중복제거하기 
    result = df[~df.name.isin(caffe)]
    print(f'{eup} {ri}의 데이터는 {result.name.count()}개 입니다')
    result.to_excel(f'naver_{eup}_{ri}.xlsx', index=False)

driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app-root > div > div._2lx2y > div._2ky45 > a:nth-child(4) > svg"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0038F243]
	(No symbol) [0x00317FD1]
	(No symbol) [0x0020D04D]
	(No symbol) [0x0023C0B0]
	(No symbol) [0x0023C22B]
	(No symbol) [0x0026E612]
	(No symbol) [0x002585D4]
	(No symbol) [0x0026C9EB]
	(No symbol) [0x00258386]
	(No symbol) [0x0023163C]
	(No symbol) [0x0023269D]
	GetHandleVerifier [0x00629A22+2655074]
	GetHandleVerifier [0x0061CA24+2601828]
	GetHandleVerifier [0x00438C0A+619850]
	GetHandleVerifier [0x00437830+614768]
	(No symbol) [0x003205FC]
	(No symbol) [0x00325968]
	(No symbol) [0x00325A55]
	(No symbol) [0x0033051B]
	BaseThreadInitThunk [0x773C6BD9+25]
	RtlGetFullPathName_UEx [0x77CE8FD2+1218]
	RtlGetFullPathName_UEx [0x77CE8F9D+1165]
